In [26]:
from langdetect import detect

In [7]:
import pandas as pd
import math

In [8]:
# load annotated metadata and relations and recreate gdf file for exporting to gephi 
annotated_metadata = pd.read_csv('annotated_videonet_seeds_elsa_spiderman_2019_08_18_metadata.csv')
annotated_rels = pd.read_csv('videonet_seeds_elsa_spiderman_2019_08_18_relations_orig.csv')
# drop redundant row index column
annotated_metadata = annotated_metadata.drop("Unnamed: 0", axis=1)
annotated_rels = annotated_rels.drop("Unnamed: 0", axis=1)

annotated_metadata.rename(columns={
    'nodeid': 'nodedef>name VARCHAR',
    'label': 'label VARCHAR',
    'subscriberCount': 'subscriberCount INT',
    'label_clean': 'labelClean VARCHAR',
    'title_language': 'titleLanguage VARCHAR',
    'url': 'url VARCHAR',
    'seed_channel': 'seedChannel BOOLEAN'
    
},
    inplace=True)

In [22]:
annotated_metadata


,nodedef>name VARCHAR,label VARCHAR,isSeed VARCHAR,seedRank INT,publishedAt INT,channelTitle VARCHAR,channelId VARCHAR,videoCategoryLabel VARCHAR,viewCount INT,likeCount INT,...,favoriteCount INT,commentCount INT,labelClean VARCHAR,titleLanguage VARCHAR,url VARCHAR,seedChannel BOOLEAN,is_inappropriate,is_live_streaming,annotator,Comments
0,Ut9iby5VChA,X,yes,1.0,1503406880,wai bom,UCir-ay37PYv8-lQOk_dxzFA,People & Blogs,2406035,2918.0,...,0,268.0,spider man and elsa have a new baby,en,https://www.youtube.com/watch?v=Ut9iby5VChA,True,YES,NO,JS,NaN
1,3ZdXn-uYz7I,X,yes,2.0,1484845230,Mathias Jost,UC96vy6lHoIKWSevWJEC47xA,People & Blogs,1763,12.0,...,0,5.0,Elsa Anna And Olaf In The Pool Frozen,en,https://www.youtube.com/watch?v=3ZdXn-uYz7I,True,YES,NO,MTO,Pregnancy
2,G_qx2ZeHJ4o,X,no,NaN,1491475843,Eve My Tube,UC0xH3QQdLAIu7srkkbTG8CQ,Entertainment,7016116,7799.0,...,0,675.0,ละครหรรษา เอลซ่าคลอดลูก เเฝงข้อคิด l พี่อีฟ ข้...,th,https://www.youtube.com/watch?v=G_qx2ZeHJ4o,False,YES,NO,MTO,Pregnancy
3,7nw-QA_-ED8,_,no,NaN,1502205344,Glamour,UCY9eEaH19ulZmdusctU1ejA,Entertainment,43156098,207189.0,...,0,7579.0,This is Your Pregnancy in 2 Minutes | Glamour,en,https://www.youtube.com/watch?v=7nw-QA_-ED8,False,NO,NO,MTO,Pregnancy
4,Jy2dTbOGTBE,_,no,NaN,1427363679,Post Hostel,UCD4MbghmkGS3aF5ymsc8_bw,People & Blogs,7880193,4354.0,...,0,53.0,The Deli Show - הריון,en,https://www.youtube.com/watch?v=Jy2dTbOGTBE,False,NO,NO,MTO,Pregnancy
5,CQ_C9KeeEbk,_,no,NaN,1531326601,Alex Ramirès,UCp_CBY2rEWs_CPI3HaC6tbg,Comedy,47904738,110190.0,...,0,2507.0,La Petite Sirene LowCost (Alex Ramirès),fr,https://www.youtube.com/watch?v=CQ_C9KeeEbk,False,NO,NO,MTO,Guy acting out the little mermaid
6,ZgkkJjKpDvw,_,no,NaN,1233984252,ClubEngendros,UCN6rjYMkYv5eUAHbFwDbQmg,Comedy,19302563,15064.0,...,0,604.0,Engendros Tv: Nuevellizos.,es,https://www.youtube.com/watch?v=ZgkkJjKpDvw,False,NO,NO,MTO,Pregnancy
7,_75wztsXeUo,X,no,NaN,1499533412,Way to Hero,UCRBhx25KHV4wUhKsO29BNgw,People & Blogs,575243,2083.0,...,0,30.0,Making Off | Ensaio de Junho | LadyBug e Cat Noir,en,https://www.youtube.com/watch?v=_75wztsXeUo,False,YES,NO,MTO,People in costumes almost having sex
8,BBm_5LSedM4,X,no,NaN,1552641301,Crafty Panda DIY,UCYDKUHScAVoI_UTVuA7ThcQ,Howto & Style,3635903,6465.0,...,0,113.0,Oh Baby Baby! Romantic Date is Over! Amazing D...,en,https://www.youtube.com/watch?v=BBm_5LSedM4,False,YES,NO,MTO,Pregnancy and smoking
9,eJqszZw6F6I,_,no,NaN,1500423727,CKN Toys,UCfaZw8XH_zmAVkBst_MPD6w,Entertainment,126942725,200689.0,...,0,NaN,McDonalds Drive Thru Prank!! Batman Payback Ti...,en,https://www.youtube.com/watch?v=eJqszZw6F6I,False,NO,NO,MTO,Child batman and grown up spiderman eating McD


In [21]:
# todo set label VARCHAR to  is_inappropriate , only label if inappropriate :(
annotated_metadata['label VARCHAR'] = annotated_metadata.is_inappropriate.apply(lambda x : 'X' if x == 'YES' else '_')

In [10]:
annotated_rels

,edgedef>node1 VARCHAR,node2 VARCHAR,directed BOOLEAN
0,Ut9iby5VChA,G_qx2ZeHJ4o,True
1,Ut9iby5VChA,7nw-QA_-ED8,True
2,Ut9iby5VChA,Jy2dTbOGTBE,True
3,Ut9iby5VChA,CQ_C9KeeEbk,True
4,Ut9iby5VChA,ZgkkJjKpDvw,True
5,Ut9iby5VChA,_75wztsXeUo,True
6,Ut9iby5VChA,BBm_5LSedM4,True
7,Ut9iby5VChA,eJqszZw6F6I,True
8,Ut9iby5VChA,nfXJ9hX5tvs,True
9,Ut9iby5VChA,9-rv1TpxxnA,True


In [4]:
def df_row_to_csv_row(row_list):
    row_list_new = []
    for x in row_list:
        if isinstance(x, float) and math.isnan(x):
            x = ''
        row_list_new.append(x)
    return ','.join((str(x) for x in row_list_new))
        


In [11]:
gdf_metadata_header = ','.join((str(x) for x in annotated_metadata.columns))
gdf_rels_header = ','.join((str(x) for x in annotated_rels.columns))

In [23]:
with open('annotated_gdf.gdf', 'w') as f:
    f.write(gdf_metadata_header + '\n')
    for row in annotated_metadata.values:
        f.write(df_row_to_csv_row(row) + '\n')
    f.write(gdf_rels_header + '\n')
    for row in annotated_rels.values:
        f.write(','.join((str(x) for x in row))+ '\n')
        

In [49]:
def parse_gdf_channel(file_name):
    with open(file_name) as f:
        rows = [line.strip().split(',') for line in f.readlines()]
        # print(rows)
        metadata = [row for row in rows if len(row) > 3]
        graph_data = [row for row in rows if len(row) == 3]
        return pd.DataFrame(metadata[1:], columns=metadata[0]), pd.DataFrame(
            graph_data[1:], columns=graph_data[0])

def parse_gdf_video(file_name):
    with open(file_name, encoding='utf-8') as f:
        rows = [line.strip().split(',') for line in f.readlines()]
        # print(rows)
        metadata = [row[:14] for row in rows if len(row) > 3]
        graph_data = [row for row in rows if len(row) == 3]
        
        return pd.DataFrame(metadata[1:], columns=metadata[0]), pd.DataFrame(
            graph_data[1:], columns=graph_data[0])
    

def all_caps_to_proper(string):
    out_string = ''
    for token in string.split(' '):
        if token.isupper():
            out_string += ' ' + token.capitalize()
        else:
            out_string += ' ' + token
    return out_string.strip()
            

In [23]:
def join_update_clean_dfs(df_graph, df_metadata, title, replace_node_ids=True):
    df_metadata.rename(columns={
        'nodedef>name VARCHAR': 'nodeid',
        'label VARCHAR': 'label',
        'subscriberCount INT': 'subscriberCount'
    },
        inplace=True)
    df_graph.rename(columns={
        'edgedef>node1 VARCHAR': 'nodeid_1',
        'node2 VARCHAR': 'nodeid_2',
        'directed BOOLEAN': 'directed'
    },
        inplace=True)
    if replace_node_ids is True:
        df_graph = pd.merge(df_graph,
                            df_metadata[['label', 'nodeid']],
                            left_on='nodeid_1',
                            right_on='nodeid',
                            how='right')[['label', 'nodeid_2', 'directed'
                                          ]].rename(columns={'label': 'node1'})

        df_graph = pd.merge(df_graph,
                            df_metadata[['label', 'nodeid']],
                            left_on='nodeid_2',
                            right_on='nodeid',
                            how='right')[['node1', 'label', 'directed'
                                          ]].rename(columns={'label': 'node2'})
    return df_metadata, df_graph


def load_video_data(filename):
    df_video_metadata, df_video_rel = parse_gdf_video(filename + '.gdf')        
    # clean up titles for language classification
    df_video_metadata['label_clean'] = df_video_metadata['label VARCHAR'].apply(all_caps_to_proper)
    # detect video title language 
    df_video_metadata['title_language'] = df_video_metadata['label_clean'].apply(detect)
    # clean column names and update node ids
    df_video_metadata, df_video_graph = join_update_clean_dfs(df_video_rel, df_video_metadata, filename)
    df_video_metadata['url'] = 'https://www.youtube.com/watch?v=' + df_video_metadata['nodeid']
    # get channel ids for seed videos
    seed_channel_ids = list(df_video_metadata[df_video_metadata['isSeed VARCHAR'] == 'yes']['channelId VARCHAR'])
    # label whether or not all videos are in the seed channel
    df_video_metadata['seed_channel'] = df_video_metadata['channelId VARCHAR'].apply(lambda x : True if x in seed_channel_ids else False)
    df_video_metadata.to_csv(filename + '_metadata.csv')
    df_video_graph.to_csv(filename + '_relations.csv')
    return df_video_metadata, df_video_graph
    

def load_channel_data(filename):
    df_metadata, df_rel = parse_gdf_channel(filename + '.gdf')        
    # clean column names and update node ids
    df_metadata, df_graph = join_update_clean_dfs(df_rel, df_metadata, filename)
    df_metadata['url'] = 'https://www.youtube.com/channel/' + df_metadata['nodeid']
    df_metadata.to_csv(filename + '_metadata.csv')
    df_graph.to_csv(filename + '_relations.csv')
    return df_metadata, df_graph
    

    
df_video_metadata, df_video_graph = load_video_data('masha_and_shark_2019_05_14')

In [8]:
df_channel_metadata, df_channel_graph = load_channel_data('channelnet_disney_2019_05_07')

FileNotFoundError: [Errno 2] No such file or directory: 'channelnet_disney_2019_05_07.gdf'

In [10]:
df_channel_graph

NameError: name 'df_channel_graph' is not defined

In [24]:
df_video_metadata

,nodeid,label,isSeed VARCHAR,seedRank INT,publishedAt INT,channelTitle VARCHAR,channelId VARCHAR,videoCategoryLabel VARCHAR,viewCount INT,likeCount INT,dislikeCount INT,dislikeLikeRatio FLOAT,favoriteCount INT,commentCount INT,label_clean,title_language,url,seed_channel
0,KYniUCGPGLs,Маша и Медведь (Masha and The Bear) - Маша плю...,yes,1,1328013326,Get Movies,UClZkHt2kNIgyrTTPnSQV3SA,Entertainment,3572394626,4511814,2450208,0.54306494017706,0,27170,Маша и Медведь (Masha and The Bear) - Маша плю...,bg,https://www.youtube.com/watch?v=KYniUCGPGLs,True
1,XqZsoesa55w,Baby Shark Dance | Sing and Dance! | Animal So...,yes,2,1466204430,Pinkfong! Kids Songs & Stories,UCcdwLMPsaU2ezNSJU1nFoBQ,Education,2762274288,7161771,2567562,0.35850936870224,0,,Baby Shark Dance | Sing and Dance! | Animal So...,en,https://www.youtube.com/watch?v=XqZsoesa55w,True
2,x1fe8-Qli9E,Маша и Медведь (Masha and The Bear) - Приятног...,no,,1348230424,Get Movies,UClZkHt2kNIgyrTTPnSQV3SA,Entertainment,1236260225,1418208,755111,0.5324402344367,0,9374,Маша и Медведь (Masha and The Bear) - Приятног...,bg,https://www.youtube.com/watch?v=x1fe8-Qli9E,True
3,_b9F05WE28Y,👶 Baby Songs | Dave and Ava | Nursery Rhymes 👶,no,,1557832469,Dave and Ava - Nursery Rhymes and Baby Songs,UC6zhI71atP7YLoZyIyCIGNw,Education,3262,308,160,0.51948051948052,0,0,👶 Baby Songs | Dave and Ava | Nursery Rhymes 👶,en,https://www.youtube.com/watch?v=_b9F05WE28Y,False
4,rKk9uJqi4hY,Surprise Eggs Wildlife Toys | Learn Wild Anima...,no,,1447556913,ChuChuTV Surprise Eggs Learning Videos,UCKcQ7Jo2VAGHiPMfDwzeRUw,Education,589326232,756885,467884,0.61817052788733,0,22364,Surprise Eggs Wildlife Toys | Learn Wild Anima...,en,https://www.youtube.com/watch?v=rKk9uJqi4hY,False
5,Bczkf9MjMBE,Couleurs Voitures Dessin Animé pour Enfants av...,no,,1557814110,Car For Kids,UCO0dv06t_DDJDthBlzY4RFA,Gaming,13137,157,70,0.44585987261146,0,1,Couleurs Voitures Dessin Animé pour Enfants av...,fr,https://www.youtube.com/watch?v=Bczkf9MjMBE,False
6,6VSaExP2anI,Mickey Mouse and Friends | Minnie s Bow-Toons ...,no,,1321036829,DisneyJuniorUK,UC7Gf2tZ8coTX2ckTPgn62iQ,Entertainment,121480020,64250,38704,0.60239688715953,0,247,Mickey Mouse and Friends | Minnie s Bow-Toons ...,en,https://www.youtube.com/watch?v=6VSaExP2anI,False
7,arqQd-nP-bc,Буба все серии Мультики для детей 🔴,no,,1544784790,KEDOO МУЛЬТИКИ для детей,UCwMABIDNcqe2GS_Sm2jHKGg,Film & Animation,44007408,68565,39448,0.57533727120251,0,0,Буба все серии Мультики для детей 🔴,ru,https://www.youtube.com/watch?v=arqQd-nP-bc,False
8,nShz5rkeX2E,Masha y el Oso - Episodios favoritos de Masha ...,no,,1490349603,Masha y el Oso,UCuSo4gcgxJRf4Bzu43wwVyg,Film & Animation,128762289,165663,93309,0.56324586660872,0,175,Masha y el Oso - Episodios favoritos de Masha ...,es,https://www.youtube.com/watch?v=nShz5rkeX2E,False
9,xIbrxsirEnA,Mascha und der Bär - Folge 2: Das Mascha Speziale,no,,1405440223,EUROPA Kinderprogramm,UCxkQwIshySkwOqEuZ28o8KA,Film & Animation,588631242,803031,387974,0.4831370146358,0,,Mascha und der Bär - Folge 2: Das Mascha Speziale,de,https://www.youtube.com/watch?v=xIbrxsirEnA,False


In [7]:
df_video_graph

,node1,node2,directed
0,Маша и Медведь (Masha and The Bear) - Маша плю...,Маша и Медведь (Masha and The Bear) - Приятног...,true
1,Маша и Медведь (Masha and The Bear) - Большая ...,Маша и Медведь (Masha and The Bear) - Приятног...,true
2,Маша и Медведь (Masha and The Bear) - Дальний ...,Маша и Медведь (Masha and The Bear) - Приятног...,true
3,Маша и Медведь (Masha and The Bear) - Новая ме...,Маша и Медведь (Masha and The Bear) - Приятног...,true
4,Маша и Медведь (Masha and The Bear) - Первая в...,Маша и Медведь (Masha and The Bear) - Приятног...,true
5,Маша и Медведь (Masha and The Bear) - Подкидыш...,Маша и Медведь (Masha and The Bear) - Приятног...,true
6,Маша и Медведь (Masha and The Bear) - Раз два...,Маша и Медведь (Masha and The Bear) - Приятног...,true
7,Маша и Медведь (Masha and The Bear) - Фокус-по...,Маша и Медведь (Masha and The Bear) - Приятног...,true
8,Маша и Медведь (Masha and The Bear) - Усатый-П...,Маша и Медведь (Masha and The Bear) - Приятног...,true
9,Маша и Медведь (Masha and The Bear) - Первый р...,Маша и Медведь (Masha and The Bear) - Приятног...,true
